<a href="https://colab.research.google.com/github/OIEIEIO/MoonWalker-HUD/blob/main/MooonWalkerWIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyQt5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.4 MB 4.9 MB/s 
     |████████████████████████████████| 344 kB 66.1 MB/s 
     |████████████████████████████████| 59.9 MB 1.3 MB/s 


In [ ]:
# clean this code and create a Jupyter notebook for running the code on Google Colab.
# each step should have its own function
# 1. Download
# 2. Convert to PNG
# 3. Circle Detection
# 4. Get circles' coordinates
# 5. Get circles from images
# 6. Display all circles on a single image to compare with the real image (NASA)
# 7. Get all circles' centre (X,Y) in an excel file
# 8. Create a function to detect circles in this old data https://asc.gsfc.nasa.gov/family2.html
# 9. Display a real image that is the same real image in (6)
# 10. Display all of the image's circles on the same image to compare our result

In [ ]:
# TODO save file names in a pickle for each orbit for converting them to png and moving them to a tmp folder
# TODO copy those files to each orbit directory and save the required data in excel sheet
# TODO Cluster the orbits and save two pngs for each cluster, one with all the circles and one with all circles and constant circle, save their coordinates in an excel sheet.
# TODO create a function to remove the frames that overlap with the moon or other satellites.
# TODO make a GUI for the user to select operation time, time for each orbit and the time frame for each observation (XX Minutes)
# TODO Create a function to find saturation points

In [ ]:
# Info from lib /tween class
'''
         * @param startValue The starting value of the animation
         * @param endValue The value that the animation should end at
         * @param duration The duration of the animation
         * @param interpolationFunction The type of interpolation (easing) to use to make this animation look nice
         * @param tweenEndListener The listener to call when tween ends
         * @param loop Boolean for whether or not to loop the animation
'''

# TODO This is for streaming the video from the camera and saving it as a video
'''
import socket
import time
import cv2
import pickle

HEADERSIZE = 10

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('192.168.1.107', 1234))
s.listen(5)

while True:
    clientsocket, address = s.accept()
    print(f"connection from {address} has been established!")

    d = {'Object': 'C19-SpySat', 'Mode': 'Standard'}
    msg = pickle.dumps(d)
    msg = bytes(f"{len(msg):<{HEADERSIZE}}", 'utf-8') + msg


    i = 0
    filename = 'live_stream'
    clientsocket.send(msg)
    while True:
        i = i+1
        ret, frame = cap.read()
        out.write(frame)
        # print(f"{i} frames from camera have been captured")
        # cv2.imshow('frame', frame)
        # if cv2.waitKey(33) & 0xFF == ord('q'):
        #     break
        if i % 5 == 0:
            data = pickle.dumps(frame)
            clientsocket.sendall(data)

    clientsocket.close()

cap.release()
out.release()
cv2.destroyAllWindows()
print("Done")
'''


class C19SpySat:
    def __init__(self):
        self.first_sat_for_each_orbit = '1'  # TODO fixed it if None
        self.total_sats_per_orbit = '1'  # TODO fixed it if None
        self.images_directory = 'TLE\\ASC Images\\'
        self.extract_sats_data()
        self.assign_nearest_sats()
        self.sats_dict = self.get_sats_circle_data()
        self.tmp_data = sorted(self.sats_dict.items(), image_matrix=asc_mat, key=operator.itemgetter(0))
        self.asc_circles_positions = asc_circles_positions
        self.short_sat_info = short_sat_info
        self.image_info = {}
        # self.screen_size = screen_size
        self.dict_circles = {}
        self.circles = []
        # self.tween_x_circles = []
        # self.tween_y_circles = []
        self.circles_center_points = []
        self.total_circles = []
        self.dict_circles_positions = {}

    def download_orbits(self):
        '''
        Function to download files from FTP according to the user input.
        Uses multi-threading to download the files.
        '''
        self.setup_FTP()
        self.list_directory()


    def setup_FTP(self):
        '''
        Function to connect to NASA's FTP.
        Uses ftplib module.
        '''
        nasa_ftp_user = 'anonymous'
        nasa_ftp_pass = 'mohamed.amin.nasif@gmail.com'
        self.ftp = FTP('asc-csr.nasa.gov')
        self.ftp.login(nasa_ftp_user, nasa_ftp_pass)
        # self.ftp.cwd('/Data/Near Earth Observations/Stars/')

    def list_directory(self):
        '''
        Display all the current directory contents.
        '''
        print(self.ftp.dir())
        # def get_file(self, filename):
        #     '''
        #     Download a file from FTP server
        #     '''
        #     local_filename = os.path.join('./', filename)
        #     file = open(local_filename, 'wb')
        #     self.ftp.retrbinary('RETR' + filename, file.write)
        #     file.close()

    def copy_file(self, new_filepath, old_full_filepath):
        '''
        Copy a file from one path to another.
        '''
        with open(new_filepath, "wb") as fwb:
            with open(old_full_filepath, 'rb') as frb:
                fwb.write(frb.read())

    def extract_sats_data(self):
        '''
        Extract information of all the satellites and save them in a dictionary.

        The dictionary items contain:
        file_name : 019E_SPY_SAT
        prop_time : 1854.88000
        inc : 51.643402
        raan : 344.195
        e : 0.0007217
        w : 295.06799
        m : 276.94342
        n : 14.945268
        n_dot : 0.00002267
        period : 1436.1202
        perigee_height : 645.539
        semimajor_axis : 6727.000
        mean_motion : 0.003380
        True_anomaly : 211.73653
        RA : 317.65082
        DEC : 33.82140
        Latitude : 8.927
        Longitude : 231.464
        Altitude : 657.091
        Velocity : 7.823774
        Visibility : 1
        Image_time : Sep  5,2010  08:30:32
        '''
        sats_data = open('sats_dat.txt', 'r')
        sats_list = sats_data.readlines()
        sat_dict = {}
        i = -1
        for line in sats_list:
            tmp_line = line.split()
            if len(tmp_line) == 3 and re.search(r"^[\d]", tmp_line[0]):
                i += 1
                sat_dict[i] = {'file_name': tmp_line[0][:3] + 'E_' + tmp_line[0][3:] + '_SPY_SAT',
                               'prop_time': float(tmp_line[1]),
                               'inc': float(tmp_line[2]),
                               'raan': float(sats_list[i + 1].split()[1]),
                               'e': float(sats_list[i + 2].split()[1]),
                               'w': float(sats_list[i + 3].split()[1]),
                               'm': float(sats_list[i + 4].split()[1]),
                               'n': float(sats_list[i + 5].split()[1]),
                               'n_dot': float(sats_list[i + 6].split()[1]),
                               'period': float(sats_list[i + 7].split()[1]),
                               'perigee_height': float(sats_list[i + 8].split()[1]),
                               'semimajor_axis': float(sats_list[i + 9].split()[1]),
                               'mean_motion': float(sats_list[i + 10].split()[1]),
                               'True_anomaly': sats_list[i + 11].split()[1],
                               'RA': sats_list[i + 12].split()[1],
                               'DEC': sats_list[i + 13].split()[1],
                               'Latitude': sats_list[i + 14].split()[1],
                               'Longitude': sats_list[i + 15].split()[1],
                               'Altitude': sats_list[i + 16].split()[1],
                               'Velocity': sats_list[i + 17].split()[1],
                               'Visibility': sats_list[i + 18].split()[1],
                               'Image_time': tmp_line[2]}
        sats_data.close()
        return sat_dict

    def assign_nearest_sats(self):
        '''
        Return the nearest sats in the TLE file to C19-SpySat.
        '''
        # satellites_theta = self.tmp_data[0][1]['theta_TLE'] - c19_theta
        satellites_y = self.tmp_data[0][1]['center_y_TLE'] - c19_y
        nearest_sats_y = list(satellites_y).sort(key=lambda x: abs(x))
        nearest_sats_y = list(satellites_y).sort()
        # nearest_sats_theta = list(satellites_theta).sort(key=lambda x: abs(x))

    def convert_to_png(self):  # TODO current state -> convert @2125 images and create directory to each satellite's name
        '''
        1. Create a folder of each sats' name
        2. Convert each image to png
        '''
        i = 0
        os.chdir("TLE\\ASC Images")
        for file in os.listdir("."):
            if re.search(r".lbl", file) and i != 2125:  # TODO To get the orbits images -> self.export_from_txt(self, 215, 2125)
                image_name = file.replace(".lbl", "")
                image_path = image_name + ".txt"
                os.rename(file, image_path)
                png_file = image_path.replace(".txt", "") + '.png'
                self.copy_file(png_file, image_path)
                os.remove(image_path)
                sat_name = image_name[5:]
                sat_folder = image_name[5:10] + 'E_' + image_name[10:-4]
                if not os.path.exists(sat_folder):
                    os.makedirs(sat_folder)
                shutil.move(png_file, sat_folder)
            i += 1

    def orbit_segmentation(self):
        '''
        Function that groups the sats together based on their coordinate to
        form an orbit. Only works if there are 10 satellites per orbit.
        '''
        s = self.sats_dict
        os.chdir("TLE\\ASC Images")
        sats_dict = s
        tmp_paths_dict = {}
        self.dict_circles_positions = {}
        for file in os.listdir("."):
            if re.search(".png", file):
                folder_name = file.replace(".png", "")
                tmp_paths_dict[folder_name] = {}
                self.dict_circles_positions[folder_name] = {}
                if not os.path.exists("TMP"):
                    os.makedirs("TMP")
                for file1 in os.listdir("."):
                    if re.search("^" + folder_name, file1) and not re.search(r"matrix", file1):
                        # [os.rename("TLE\\ASC Images\\TMP\\" + file, "TLE\\ASC Images\\TMP\\" + file.replace(" ", "__")) for file in os.listdir("TLE\\ASC Images\\TMP") if re.search(" ", file)]
                        # print("TLE\\ASC Images\\TMP\\" + file)
                        # [os.rename("TLE\\ASC Images\\TMP\\" + file, "TLE\\ASC Images\\TMP\\" + file.replace(" ", "__")) for file in os.listdir("TLE\\ASC Images\\TMP\\" + folder_name) if re.search(" ", file)]
                        [os.rename(".\\TMP\\" + file, "TMP\\" + file.replace(" ", "__")) for file in os.listdir("TMP") if re.search(" ", file)]
                        shutil.move(".\\" + file1, "TMP")
                        tmp_paths_dict[folder_name][file1] = folder_name + "\\" + file1
                        image_path = "./TMP" + file1[file1.find(folder_name):]
                        img1 = cv2.imread(image_path, 1)
                        doc = {'file': file, 'image_matrix': img1, 'date': sats_dict[file1]['Image_time']}
                        self.i2mage_info[file1] = doc
                        self.d2222222222222222
 

In [ ]:
import os
import sys
import cv22
import time
import pickle
import socket
import datetime
import numpy as np
from ftplib import FTP
from PyQt5 import QtWidgets
from MainWindow import Ui_MainWindow
from PyQt5.QtCore import *
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from Thread_FTP_Downloader import QThreadFtpDownloader
from Thread_Cleaning_FITS import QThreadCleaning
from Thread_Orbit_Clustering import QThreadOrbitCluster
from Thread_Convert2PNG import QThreadPngConvert
from Thread_Detection import QThreadDetection
from Thread_Copy import QThreadCopy


class MainWindow(QtWidgets.QMainWindow, Ui_MainWindow):
    def __init__(self, *args, obj=None, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)
        self.setupUi(self)

        self.ftp = FTP('ftp.asc-csa.gc.ca')
        self.file_list = []
        self.number_of_files = 0
        self.thread_ftp_downloader = QThreadFtpDownloader()
        self.thread_cleaning = QThreadCleaning()
        self.thread_orbit_cluster = QThreadOrbitCluster()
        self.thread_png_convert = QThreadPngConvert()
        self.thread_detection = QThreadDetection()
        self.thread_copy = QThreadCopy()
        self.ftp_selected_year = 0
        self.ftp_selected_doy = 0
        self.all_orbits = []
        self.orbit_first_sample_num = []
        self.all_orbits_backup = []
        self.all_orbits_data = []
        self.all_orbits_file_names = []
        self.all_orbits_number = 0

        # self.thread_ftp_downloader.finished.connect(self.cleaning_function())
        self.thread_orbit_cluster.finished.connect(self.orbit_cluster_plot)
        self.thread_orbit_cluster.file_names.connect(self.orbit_cluster_save_file_names)
        self.thread_orbit_cluster.backup.connect(self.orbit_cluster_backup)
        # self.thread_png_convert.data_to_save.connect(self.save_png_converted_function)
        self.txt_download_addr.setText(os.getcwd())
        self.pb_change_download_addr.clicked.connect(self.on_change_addr_click)
        self.pb_download.clicked.connect(self.on_download_click)
        self.pb_load_cleaned.clicked.connect(self.on_load_cleaned_click)
        self.pb_convert_png.clicked.connect(self.save_png_function)
        self.pb_detect.clicked.connect(self.on_detect_click)
        self.cb_year.currentIndexChanged.connect(self.on_year_selected)
        self.cb_orbit_number.currentIndexChanged.connect(self.on_orbit_selected)
        self.rb_ftp.toggled.connect(self.set_download_mode_ftp)
        self.rb_doy.toggled.connect(self.set_download_mode_doy)
        self.rb_doy.setFocus(True)
        self.rb_doy.click()
        self.rb_local.toggled.connect(self.set_download_mode_local)
        self.pb_load_cleaned.setVisible(False)
        self.pb_detect.setVisible(False)
        self.lbl_cleaned_addr.setVisible(False)
        self.pb_detect.setEnabled(True)
        # self.pbar_download.setEnabled(False)
        self.tb_orbits.setItem(0, 0, QtWidgets.QTableWidgetItem(''))
        self.tb_orbits.setItem(0, 1, QtWidgets.QTableWidgetItem(''))
        self.tb_orbits.setItem(0, 2, QtWidgets.QTableWidgetItem(''))
        self.tb_orbits.setItem(0, 3, QtWidgets.QTableWidgetItem(''))

        self.initialize()

        # self.lbl_cleaned_addr.setText('Survey_Cleaned_2019_292')
        # self.on_detect_click()

    def initialize(self):
        print(self.ftp.login())
        self.ftp.cwd('users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/')
        year_list = sorted(self.ftp.nlst())
        print(year_list)
        self.cb_year.clear()
        self.cb_year.addItems(year_list)
        self.cb_year.setCurrentIndex(2)

    def set_all_widgets_enable(self, state):
        self.chb_raw.setEnabled(state)
        # self.chb_auto.setEnabled(state)
        self.rb_ftp.setEnabled(state)
        self.rb_doy.setEnabled(state)
        self.rb_local.setEnabled(state)
        self.pb_download.setEnabled(state)
        self.pb_change_download_addr.setEnabled(state)
        self.pb_convert_png.setEnabled(state)
        self.cb_year.setEnabled(state)
        self.cb_doy.setEnabled(state)
        self.pb_detect.setEnabled(state)
        self.cb_orbit_number.setEnabled(state)

    def orbit_cluster_save_file_names(self, file_list):
        self.all_orbits_file_names = file_list

    def on_detect_click(self):
        self.thread_detection.on_input(self.lbl_cleaned_addr.text() + '_Orbit_' + self.cb_orbit_number.currentText() + '_png')
        self.thread_detection.progress.connect(self.detection_progress_fn)
        self.thread_detection.start()
        self.set_all_widgets_enable(False)

    def save_png_function(self):
        matplotlib.use('Agg')
        self.thread_png_convert.on_input(self.lbl_cleaned_addr.text() + '_Orbit_' + self.cb_orbit_number.currentText())  # , self.all_orbits_file_names[sum(self.orbit_first_sample_num[0:t]):sum(self.orbit_first_sample_num[0:t+1])])
        self.thread_png_convert.progress.connect(self.convert_progress_fn)
        self.thread_png_convert.start()
        self.set_all_widgets_enable(False)

    def orbit_cluster_backup(self, backup):
        self.all_orbits_backup = list(backup)
        print('Orbits Backup Data', self.all_orbits_backup)

    def orbit_cluster_plot(self, orbits):
        orbits = list(orbits)
        self.all_orbits = orbits
        print('Orbits', orbits)
        with open(self.lbl_cleaned_addr.text().split('/')[-1] + '-Orbits.pickle', 'wb') as f:
            pickle.dump(orbits, f)

        # plt.plot(orbits, 'bo')
        # plt.ylabel('Number of orbits')
        # plt.show()

        X = np.array([[i, orbits[i]] for i in range(len(orbits))])
        X = StandardScaler().fit_transform(X)

        # #############################################################################
        # Compute DBSCAN
        db = DBSCAN(eps=0.3, min_samples=10).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        if n_clusters_:
            print('Estimated number of orbits: %d' % n_clusters_)
        else:
            print('Estimated number of clusters: 1')
        print('Estimated number of noise points: %d' % n_noise_)

        # Black removed and is used for noise instead.
        unique_labels = set(labels)
        colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
        legend_number = 0
        legend_plot = []
        self.orbit_first_sample_num = []
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]
            legend_number += 1
            class_member_mask = (labels == k)

            xy = X[class_member_mask & core_samples_mask]
            self.orbit_first_sample_num.append(len(xy[:, 0]))
            l, = plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=14, label=str(legend_number))
            legend_plot.append(l)

            xy = X[class_member_mask & ~core_samples_mask]
            plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=6)
        plt.legend(handles=legend_plot)
        self.all_orbits_number = legend_number

        for i in range(legend_number):
            if not os.path.isdir(self.lbl_cleaned_addr.text() + '_Orbit_' + str(i+1)):
                os.mkdir(self.lbl_cleaned_addr.text() + '_Orbit_' + str(i+1))
            tmp_arr = self.all_orbits_file_names[sum(self.orbit_first_sample_num[0:i]):sum(self.orbit_first_sample_num[0:i+1])]

            # tmp_file_list = []
            # for j in tmp_arr:
            #     tmp_file_list.append(j + ' ' + self.lbl_cleaned_addr.text() + '_Orbit_' + str(i+1))
            # self.thread_copy.on_input()
            # self.thread_copy.progress.connect(self.copy_progress_fn)
            # self.thread_copy.start()

            for c, j in enumerate(tmp_arr, 0):
                os.system('cp ' + j + ' ' + self.lbl_cleaned_addr.text() + '_Orbit_' + str(i+1))

                # value = 80.0 + ((i * c * 100) / (legend_number * len(tmp_arr)))
                # self.pbar_orbit.setValue(int(value))
                # tmp = "%.1f" % value
                # self.lbl_orbit.setText(tmp + '%')
                # print("%f%% Read" % value)
                # time.sleep(0.1)

            self.all_orbits_data.append(self.all_orbits_backup[sum(self.orbit_first_sample_num[0:i])])
            print(i + 1, 'th Orbit Sample Data', self.all_orbits_backup[sum(self.orbit_first_sample_num[0:i])])

        if legend_number == 1:
            if not os.path.isdir(self.lbl_cleaned_addr.text() + '_Orbit_1'):
                os.mkdir(self.lbl_cleaned_addr.text() + '_Orbit_1')
            for j in self.all_orbits_file_names:
                os.system('cp ' + j + ' ' + self.lbl_cleaned_addr.text() + '_Orbit_1')

        self.cb_orbit_number.clear()
        self.cb_orbit_number.addItems([str(i + 1) for i in range(legend_number)])
        # 0 temp - 1 ra - 2 dec - 3 rol
        self.tb_orbits.setItem(0, 0, QtWidgets.QTableWidgetItem(self.all_orbits_data[0][0]))
        self.tb_orbits.setItem(0, 1, QtWidgets.QTableWidgetItem(self.all_orbits_data[0][1]))
        self.tb_orbits.setItem(0, 2, QtWidgets.QTableWidgetItem(self.all_orbits_data[0][2]))
        self.tb_orbits.setItem(0, 3, QtWidgets.QTableWidgetItem(self.all_orbits_data[0][3]))
        self.tb_orbits.setItem(0, 4, QtWidgets.QTableWidgetItem(self.all_orbits_data[0][4]))

        value = 100.0
        self.pbar_orbit.setValue(int(value))
        tmp = "%.1f" % value
        self.lbl_orbit.setText(tmp + '%')
        print("%f%% Read" % value)

        self.pb_convert_png.setEnabled(True)
        self.cb_orbit_number.setFocus(True)
        self.set_all_widgets_enable(True)

        if n_clusters_ == 0:
            n_clusters_ = 1
        plt.title('Estimated Number of Orbits: %d' % n_clusters_)
        plt.show()

    def on_orbit_selected(self):
        self.tb_orbits.setItem(0, 0, QtWidgets.QTableWidgetItem(self.all_orbits_data[self.cb_orbit_number.currentIndex()][0]))
        self.tb_orbits.setItem(0, 1, QtWidgets.QTableWidgetItem(self.all_orbits_data[self.cb_orbit_number.currentIndex()][1]))
        self.tb_orbits.setItem(0, 2, QtWidgets.QTableWidgetItem(self.all_orbits_data[self.cb_orbit_number.currentIndex()][2]))
        self.tb_orbits.setItem(0, 3, QtWidgets.QTableWidgetItem(self.all_orbits_data[self.cb_orbit_number.currentIndex()][3]))
        self.tb_orbits.setItem(0, 4, QtWidgets.QTableWidgetItem(self.all_orbits_data[self.cb_orbit_number.currentIndex()][4]))

    def set_download_mode_ftp(self):
        if self.rb_ftp.isChecked():
            self.txt_download_addr.setText('users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/')
            # self.chb_raw.setVisible(False)
            self.pb_download.setVisible(True)
            # self.chb_auto.setVisible(True)
            self.cb_year.setVisible(False)
            self.cb_doy.setVisible(False)
            self.lbl_year.setVisible(False)
            self.lbl_doy.setVisible(False)
            self.pb_change_download_addr.setVisible(False)

    def set_download_mode_doy(self):
        if self.rb_doy.isChecked():
            self.txt_download_addr.setText(os.getcwd())
            self.pb_download.setVisible(True)
            # self.chb_auto.setVisible(True)
            self.cb_year.setVisible(True)
            self.cb_doy.setVisible(True)
            self.lbl_year.setVisible(True)
            self.lbl_doy.setVisible(True)
            self.pb_change_download_addr.setVisible(True)

    def set_download_mode_local(self):
        if self.rb_local.isChecked():
            self.txt_download_addr.setText('')
            self.pb_download.setVisible(False)
            # self.chb_auto.setVisible(False)
            self.cb_year.setVisible(False)
            self.cb_doy.setVisible(False)
            self.lbl_year.setVisible(False)
            self.lbl_doy.setVisible(False)
            self.pb_change_download_addr.setVisible(True)

    def detection_progress_fn(self, value):
        if value == 100:
            self.set_all_widgets_enable(True)
        self.pbar_detect.setValue(int(value))
        tmp = "%.1f" % value
        self.lbl_detect.setText(tmp + '%')
        print("%f%% Detection Completed" % value)

    def convert_progress_fn(self, value):
        if value == 100:
            self.pb_detect.setEnabled(True)
            self.set_all_widgets_enable(True)
            self.on_detect_click()
        self.pbar_convert_png.setValue(int(value))
        tmp = "%.1f" % value
        self.lbl_convert.setText(tmp + '%')
        print("%f%% Converted" % value)

    def orbit_progress_fn(self, value):
        # if value == 80:
        #     self.set_all_widgets_enable(True)
        self.pbar_orbit.setValue(int(value))
        tmp = "%.1f" % value
        self.lbl_orbit.setText(tmp + '%')
        print("%f%% Read" % value)

    def cleaning_progress_fn(self, value):
        if value == 100:
            self.set_all_widgets_enable(True)
            self.orbit_clustering_function()
        self.pbar_cleaning.setValue(int(value))
        tmp = "%.1f" % value
        self.lbl_cleaning.setText(tmp + '%')
        print("%f%% Cleaning" % value)

    def download_progress_fn(self, value):
        if value == 100:
            self.set_all_widgets_enable(True)
            if not os.path.isdir(self.lbl_cleaned_addr.text()):
                os.mkdir(self.lbl_cleaned_addr.text())
            # self.orbit_clustering_function()
            if self.rb_doy.isChecked():
                self.thread_cleaning.on_input(self.txt_download_addr.text() + '/Survey_' +
                    self.cb_year.currentText() + '_' + self.cb_doy.currentText() + '_AutoDownload', self.lbl_cleaned_addr.text())
            elif self.rb_ftp.isChecked():
                self.thread_cleaning.on_input('./Survey_' + str(self.ftp_selected_year) + '_' + str(self.ftp_selected_doy) + '_AutoDownload', self.lbl_cleaned_addr.text())
            self.thread_cleaning.progress.connect(self.cleaning_progress_fn)
            self.thread_cleaning.start()
            self.set_all_widgets_enable(False)
        self.pbar_download.setValue(int(value))
        tmp = "%.1f" % value
        self.lbl_download.setText(tmp + '%')
        print("%f%% Downloaded" % value)

    def print_output(self, s):
        print(s)

    @pyqtSlot()
    def on_change_addr_click(self):
        folder_addr = str(QtWidgets.QFileDialog.getExistingDirectory(self, "Select Directory"))
        self.txt_download_addr.setText(folder_addr)
        if len(folder_addr) > 0 and self.rb_local.isChecked():
            self.lbl_cleaned_addr.setText(folder_addr)
            self.pbar_download.setValue(100.0)
            self.lbl_download.setText('100.0%')
            self.pbar_cleaning.setValue(100.0)
            self.lbl_cleaning.setText('100.0%')
            # self.orbit_clustering_function()
            self.thread_cleaning.on_input(self.txt_download_addr.text(), self.lbl_cleaned_addr.text())
            self.thread_cleaning.progress.connect(self.cleaning_progress_fn)
            self.thread_cleaning.start()
            self.set_all_widgets_enable(False)

    @pyqtSlot()
    def on_download_click(self):
        should_start = False
        if self.rb_ftp.isChecked():
            try:
                print('Start to download ...')
                self.lbl_download.setText('0.0%')
                self.pbar_download.setValue(0)
                if self.txt_download_addr.text().find('ftp://ftp.asc-csa.gc.ca/') >= 0:
                    self.txt_download_addr.setText(self.txt_download_addr.text()[len('ftp://ftp.asc-csa.gc.ca/'):])
                self.ftp.cwd('/')
                self.ftp.cwd(self.txt_download_addr.text())
                should_start = True
                tmp = self.txt_download_addr.text().split('/')
                found = False
                for i, folders in enumerate(tmp, 0):
                    # print(folders, i, '20' in folders, len(folders))
                    if ('20' in folders) and len(folders) == 4:
                        found = True
                        break
                if found:
                    self.ftp_selected_year = tmp[i]
                    self.ftp_selected_doy = tmp[i + 1]
                    print('Year', self.ftp_selected_year, ', Day of Year', self.ftp_selected_doy)
                else:
                    self.ftp_selected_year = str(time.asctime().split(' ')[-1])
                    self.ftp_selected_doy = str(datetime.datetime.now().timetuple().tm_yday)
                    print('Year', self.ftp_selected_year, ', Day of Year', self.ftp_selected_doy)
            except Exception as e:
                print('ERROR, on_download_click True ::', e)
                self.txt_download_addr.setText('users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/')
        elif self.rb_doy.isChecked():
            try:
                print('Start to download ...')
                self.lbl_download.setText('0.0%')
                self.pbar_download.setValue(0)
                self.cb_year.setEnabled(False)
                self.cb_doy.setEnabled(False)
                self.pbar_download.setEnabled(True)
                self.ftp.cwd(self.cb_doy.currentText())
                should_start = True
            except Exception as e:
                print('ERROR, on_download_click False ::', e)

        if should_start:
            if self.rb_ftp.isChecked():
                self.lbl_cleaned_addr.setText('./Survey_' + str(self.ftp_selected_year) + '_' + str(self.ftp_selected_doy) + '_Cleaned')
                print('ftp', self.lbl_cleaned_addr.text())
            elif self.rb_doy.isChecked():
                self.lbl_cleaned_addr.setText(self.txt_download_addr.text() + '/Survey_' + self.cb_year.currentText() + '_' + self.cb_doy.currentText() + '_Cleaned')
                print('doy', self.lbl_cleaned_addr.text())
            all_file_list = self.ftp.nlst()
            # print(len(all_file_list), sorted(all_file_list))
            self.file_list = []
            for file in all_file_list:
                if self.chb_raw.isChecked():
                    if file.find('clean') < 0 and file.find('cor') < 0:
                        self.file_list.append(file)
                else:
                    self.file_list.append(file)
            self.file_list = sorted(self.file_list)
            print(self.file_list)
            self.number_of_files = len(self.file_list)
            print('Going to download', self.number_of_files, 'File(s)')

            if self.rb_ftp.isChecked():
                tmp_folder = './Survey_' + str(self.ftp_selected_year) + '_' + str(self.ftp_selected_doy) + '_AutoDownload'
                if not os.path.isdir(tmp_folder):
                    os.mkdir(tmp_folder)
                self.thread_ftp_downloader.on_input(self.file_list, self.ftp, tmp_folder, self.number_of_files)
            elif self.rb_doy.isChecked():
                tmp_folder = self.txt_download_addr.text() + '/Survey_' + \
                                 self.cb_year.currentText() + '_' + self.cb_doy.currentText() + '_AutoDownload'
                if not os.path.isdir(tmp_folder):
                    os.mkdir(tmp_folder)
                self.thread_ftp_downloader.on_input(self.file_list, self.ftp, tmp_folder, self.number_of_files)
            self.thread_ftp_downloader.progress.connect(self.download_progress_fn)
            self.thread_ftp_downloader.start()
            self.set_all_widgets_enable(False)

    @pyqtSlot()
    def on_year_selected(self):
        try:
            try:
                self.ftp.cwd(self.cb_year.currentText())
            except Exception as e:
                print('ERROR, on_year_selected, ftp.cwd ::', e)
                # self.ftp.cwd('..')
                self.ftp.cwd('/')
                self.ftp.cwd('users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/')
                self.ftp.cwd(self.cb_year.currentText())
            day_list = sorted(self.ftp.nlst())
            print(day_list)
            self.cb_doy.clear()
            self.cb_doy.addItems(day_list)
            self.cb_doy.setCurrentText('306')  # TODO remove this line
        except Exception as e:
            print('ERROR, on_year_selected ::', e)
            self.rb_ftp.setChecked(True)

    def on_load_cleaned_click(self):
        pass
        # folder_addr = str(QtWidgets.QFileDialog.getExistingDirectory())
        # if len(folder_addr):
        #     self.lbl_cleaned_addr.setText(folder_addr)
        #     self.orbit_clustering_function()

    def orbit_clustering_function(self):
        matplotlib.use('MacOSX')
        self.lbl_orbit.setText('0.0%')
        self.pbar_orbit.setValue(0)

        file_list = os.listdir(self.lbl_cleaned_addr.text())
        file_list = sorted(file_list)
        print(len(file_list), file_list)

        if not os.path.isdir(self.lbl_cleaned_addr.text() + '_Dark/'):
            os.mkdir(self.lbl_cleaned_addr.text() + '_Dark/')
        self.thread_orbit_cluster.on_input(self.lbl_cleaned_addr.text(), file_list, self.lbl_cleaned_addr.text() + '_Dark/')
        self.thread_orbit_cluster.progress.connect(self.orbit_progress_fn)
        self.thread_orbit_cluster.start()
        self.set_all_widgets_enable(False)

        print('Orbit Cluster Started')


def is_connected():
    try:
        # connect to the host -- tells us if the host is actually
        # reachable
        socket.create_connection(("www.google.com", 80))
        return True
    except OSError:
        pass
    return False


if not is_connected():
    print('Connection to Internet is NOT Established!!!')
    exit(0)

app = QtWidgets.QApplication(sys.argv)

window = MainWindow()
window.show()
app.exec()

ModuleNotFoundError: ignored